# **Equipo C - 2024 - 1**

# **RandomForestRegressor**

Se importan varias bibliotecas de Python que se utilizan para cargar y procesar datos, entrenar un modelo de RandomForestRegressor, evaluar el modelo y visualizar los resultados.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go

Se carga los datos financieros de la empresa con el símbolo "FSM" desde Yahoo Finance utilizando su API de descarga de datos históricos.

In [2]:
# Cargar los datos
data = pd.read_csv('https://query1.finance.yahoo.com/v7/finance/download/FSM?period1=1597123200&period2=1628659200&interval=1d&events=history%27')


Se eliminan las filas que contienen valores nulos en el DataFrame data

In [3]:
# Preprocesamiento de datos
data.dropna(inplace=True)  # Eliminar filas con valores NaN

Se aplica la normalización de características a las columnas del DataFrame data, excluyendo las columnas 'Date' y 'Close'.

In [4]:
# Normalización de las variables
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.drop(columns=['Date', 'Close']))  # Escalar todas las características excepto la fecha y el precio de cierre

crea un nuevo DataFrame llamado scaled_df utilizando los datos normalizados (scaled_data).

In [5]:
# Convertir los datos normalizados de vuelta a un DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=data.columns[1:-1])  # Excluir la columna de fecha y el precio de cierre

Se agrega la columna de precios de cierre ('Close') del DataFrame original data al nuevo DataFrame scaled_df.

In [6]:
# Agregar la columna de precio de cierre nuevamente
scaled_df['Close'] = data['Close']

se divide el conjunto de datos en características (variables independientes) y la variable objetivo (variable dependiente) que se está tratando de predecir. Las características (X) se utilizarán para entrenar el modelo, mientras que la variable objetivo (y) se utilizará para evaluar la precisión del modelo durante el entrenamiento y para hacer predicciones una vez que el modelo esté entrenado.

In [7]:
# Separar características (X) y variable objetivo (y)
X = scaled_df.drop(columns=['Close'])  # Usaremos todas las características excepto el precio de cierre
y = scaled_df['Close']

Se divide los datos en conjuntos de entrenamiento y prueba utilizando la función train_test_split de Scikit-learn. Especificamos el tamaño del conjunto de prueba. En este caso, se establece en 0.2, lo que significa que el 20% de los datos se reservarán para el conjunto de prueba.

In [8]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Se entrena un modelo de regresión RandomForestRegressor utilizando los datos de entrenamiento (X_train, y_train). Se especifica el número de árboles en el bosque. En este caso, se utilizan 100 árboles.

In [9]:
# Entrenar el modelo RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

Se utiliza el modelo RandomForestRegressor entrenado para realizar predicciones sobre el conjunto de prueba (X_test).

In [10]:
# Realizar predicciones en el conjunto de prueba
rf_predictions = rf_model.predict(X_test)

se calculan dos métricas comunes para evaluar el rendimiento del modelo RandomForestRegressor en el conjunto de prueba.Calcula el error porcentual absoluto medio (MAPE).Calcula la raíz cuadrada del error cuadrático medio (RMSE)

In [11]:
# Calcular métricas de evaluación
mape_rf = mean_absolute_percentage_error(y_test, rf_predictions)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_predictions))

print(f'MAPE Random Forest: {mape_rf}')
print(f'RMSE Random Forest: {rmse_rf}')

MAPE Random Forest: 0.013268335327451263
RMSE Random Forest: 0.1174103020165096


Se utiliza Plotly para crear un gráfico que muestra tanto los precios de cierre reales como las predicciones del modelo RandomForestRegressor en el tiempo.

In [12]:
# Visualización del precio de cierre real y las predicciones
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'].iloc[-len(y_test):], y=y_test, mode='lines', name='Actual Price'))
fig.add_trace(go.Scatter(x=data['Date'].iloc[-len(y_test):], y=rf_predictions, mode='lines', name='Predicted Price'))
fig.update_layout(title='Predicción del Precio de Cierre Diario con RandomForestRegressor',
                  xaxis_title='Fecha',
                  yaxis_title='Precio de Cierre',
                  template='plotly_dark')
fig.show()